In [126]:
import os
import pprint
import gradio as gr
from typing import TypedDict, Optional

from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description, StructuredTool
from langgraph.graph import add_messages
from langchain_openai import ChatOpenAI
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.prebuilt import create_react_agent
from langgraph.types import RetryPolicy
from langtrace_python_sdk import get_prompt_from_registry
from sqlalchemy.sql.annotation import Annotated

from tools.tools import get_validator_questions_per_difficulty

load_dotenv()

LITE_LLM_API_KEY = os.getenv("OPENAI_API_KEY")
LITE_LLM_URL = os.getenv("OPENAI_BASE_URL")
LITE_MODEL = os.getenv("OPENAI_MODEL")

In [174]:
class MatrixValidationState(TypedDict):
    skill_id: int
    difficulty: int
    messages: Annotated[list, add_messages]


async def grading_agent(state: MatrixValidationState):
    model = ChatOpenAI(
        model=LITE_MODEL,
        base_url=LITE_LLM_URL,
        api_key=LITE_LLM_API_KEY,
        temperature=0,
        max_tokens=300,
    )
    tools = [
        StructuredTool.from_function(
            name="get_validator_questions_per_difficulty",
            func=get_validator_questions_per_difficulty,
            coroutine=get_validator_questions_per_difficulty,
        )
    ]
    intermediate_steps = []
    # prompt_id = "cmdd7k7z70082yrs5rjo3a0ti"
    # grading_prompt = get_prompt_from_registry(prompt_id)
    prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
Skill Id: 1
Difficulty Level: 4
Response of the tool get_validator_questions_per_difficulty will return question and the answer.
You validate the user knowledge against the answer that is provided!

You are grading users knowledge based on question (answer holds the answer against which you validate did the user answer correctly). If the user has fully answered the question just return "correct".
Validate the answer by comparing similarity between the answer returned from the tool and users answer!
If the user has partially answered the question, explain the correctness of the answer and explain to the user the values that should be explained additionally to complete the grading. Do not however explain the response in any way just explain what is needed by the answer. Additionally everything you ask the user has to be present in the answer! Do not ask an explanations that are not present in the answer!
Ask at most 2 additional times for additional explanations to the answer before concluding whether the answer is correct or incorrect.
Additionally if some explanation is not present in the answer, do not ask for them, conclude the grading as correct!
If you conclude after 2 additional questions that the user does not know the answer return the value "wrong"!
If the answer does not contain the explanation for a specific topic do not ask the user for it!
        """,
            ),
            (
                "ai",
                "Please explain how you would configure a custom AuthenticationProvider in a Spring Boot application using Spring Security. What are the key interfaces and how do they interact?",
            ),
            (
                "human",
                "our own auth provider implementation extends AuthenticationProvider interface, implementing authenticate and supports methods. We register it in SecurityFilterChain. Our provider talks with AuthenticationManager to provide authentication ",
            ),
            (
                "ai",
                "Your answer partially covers the configuration of a custom AuthenticationProvider in a Spring Boot application using Spring Security. \n\nTo complete the grading, please explain the following additional points:\n1. What is the purpose of the `authenticate` method in your custom AuthenticationProvider implementation?\n2. Can you describe the role of the `supports` method and what it checks for?",
            ),
            (
                "human",
                "authenticate actually runs the authentication logic while supports is used by authenentication manager to check whether auth provider can handle this auth type",
            ),
        ]
    )
    prompt = await prompt_template.ainvoke({})
    agent = create_react_agent(model=model, tools=tools)
    response = await agent.ainvoke(prompt)

    pprint.pprint(response, indent=4)
    return {
        "skill_id": state["skill_id"],
        "difficulty": state["difficulty"],
        "messages": response["messages"][-1].content,
    }


async def get_graph():
    graph = StateGraph(MatrixValidationState)
    graph.add_node("grading", grading_agent, retry_policy=RetryPolicy(max_attempts=3))
    graph.add_edge(START, "grading")
    graph.add_edge("grading", END)
    return graph.compile()

In [175]:
grading_graph = await get_graph()

response = await grading_graph.ainvoke(
    input={"skill_id": 1, "difficulty": 4, "messages": []}
)
# A custom AuthenticationProvider implements the AuthenticationProvider interface, overriding the authenticate() and supports() methods. It is registered in a SecurityFilterChain or WebSecurityConfigurerAdapter configuration. It interacts with AuthenticationManager to perform authentication logic.

{   'messages': [   SystemMessage(content='\nSkill Id: 1\nDifficulty Level: 4\nResponse of the tool get_validator_questions_per_difficulty will return question and the answer.\nYou validate the user knowledge against the answer that is provided!\n\nYou are grading users knowledge based on question (answer holds the answer against which you validate did the user answer correctly). If the user has fully answered the question just return "correct".\nValidate the answer by comparing similarity between the answer returned from the tool and users answer!\nIf the user has partially answered the question, explain the correctness of the answer and explain to the user the values that should be explained additionally to complete the grading. Do not however explain the response in any way just explain what is needed by the answer. Additionally everything you ask the user has to be present in the answer! Do not ask an explanations that are not present in the answer!\nAsk at most 2 additional times 